In [16]:
#!pip install -Uq agentstack agentops --user
#!pip install  -Uq tavily-python
# !pip install scrapegraph_py
#!pip install --upgrade jupyter ipywidgets --user


In [1]:
from crewai import Agent , Task , Crew , Process , LLM
from crewai.tools import tool
from tavily import TavilyClient
from pydantic import BaseModel , Field
from typing import List
import agentops
import os 
from dotenv import load_dotenv
import json
from scrapegraph_py import Client
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource


In [2]:
load_dotenv(override=True)
agentops.init(api_key=os.getenv("AGENTOPS_API_KEY") ,
skip_auto_end_session=True
)


🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=10d2e12e-3371-4b52-aeb2-0ab3ac15665a


In [3]:
os.environ["GEMINI_API_KEY"] = os.getenv("GOOGLE_API_KEY")
llm=LLM(
        model="gemini/gemini-1.5-pro",
        temperature=0,
        # verbose=True,    
        )


tavily_client = TavilyClient(api_key=os.getenv('TAVILY_API_KEY'))
scrape_client = Client(api_key=os.getenv("ScrapeGraph_API_KEY"))



In [4]:
output_dir = "./ai-agent-output"
os.makedirs(output_dir, exist_ok=True)

In [5]:
class SuggestedSearchQueries(BaseModel):
    queries: List[str] = Field(..., description="A list of suggested search queries."
                              ,min_items=1 , max_items=5
                                )


class SingleSearshResult(BaseModel):
    title: str = Field(..., description="Title of the page")
    content: str = Field(..., description="Content of the page")
    url: str = Field(..., description="URL of the page")
    score: float = Field(..., description="Score of the page")
    searsh_query: str = Field(... , description="the used query to get the  result")



class allSearshResults(BaseModel):
    results : List[SingleSearshResult] = Field(..., description="the list of results of the search engine")

In [6]:
class ProductSpec(BaseModel):
    specification_name: str
    specification_value: str

class SingleExtractedProduct(BaseModel):
    page_url: str = Field(..., title="The original url of the product page")
    product_title: str = Field(..., title="The title of the product")
    product_image_url: str = Field(..., title="The url of the product image")
    product_url: str = Field(..., title="The url of the product")
    product_current_price: float = Field(..., title="The current price of the product")
    product_original_price: float = Field(title="The original price of the product before discount. Set to None if no discount", default=None)
    product_discount_percentage: float = Field(title="The discount percentage of the product. Set to None if no discount", default=None)

    product_specs: List[ProductSpec] = Field(..., title="The specifications of the product. Focus on the most important specs to compare.", min_items=1, max_items=5)

    agent_recommendation_rank: int = Field(..., title="The rank of the product to be considered in the final procurement report. (out of 5, Higher is Better) in the recommendation list ordering from the best to the worst")
    agent_recommendation_notes: List[str]  = Field(..., title="A set of notes why would you recommend or not recommend this product to the company, compared to other products.")


class AllExtractedProducts(BaseModel):
    products: List[SingleExtractedProduct]

-- Field(..., description="A list of suggested search queries."):

- Field(...) is used to provide metadata or constraints on the field.
- ... (Ellipsis) means this field is required.
- description="A list of suggested search queries." adds a human-readable explanation, useful for documentation or API schemas.

# Setup Agent

### Agent 1

In [7]:
search_queries_recommender_agent = Agent(
    role="Search Queries Recommender Agent",
    goal="To Recommend 4 search queries for a given topic to be passed to  the  search engine.",
    backstory="""You're a skilled search query recommender who excels at suggesting
    relevant queries based on a given topic or keyword. You have a deep understanding
    of various search engines and their search algorithms, making you a valuable
    resource for anyone looking to optimize their search engine usage.""",
    llm=llm,
    verbose=True,    
)
search_queries_recommender_task = Task(
    description="\n".join([
        "we are looking  to  buy {product_name} at the  best prices (value for  a price strategy).",
    "we target any if  these websites to  bue from : {websites} ." ,
    "we want  to  reach all the available products on the internet to  be  compared later in another stage."
    "the stores must  sell the  product in {country}",  
    "the search keywords must be  in {language} language"
    "the search query must an ecommerse webpage for  product  , and not a blog or listing page.",
    
    ]),
    expected_output="a JSON object containing a list of suggested search queries.",
    output_json = SuggestedSearchQueries,
    output_file = os.path.join(output_dir , "step1_suggested_query.json"),
    agent=search_queries_recommender_agent,
  
)

### Agent 2

In [8]:
@tool
def search_engin_tool(query: str) -> str:
    """
    Useful for searsh-based queries . Use this  to  find current  information about any query related pages using a searsh engine.
    """
    result = tavily_client.search(query)
    # print("this is the ffffffffffffffffff: ")
    # print(result)
    return result


search_engine_agent = Agent(
    role="Search Engine Agent",
    goal="To find current information about any query related pages using a searsh engine.",
    backstory="\n".join(["The agent is  designed to  help inlooking for  products by searshing for  products based on the seggested searsh queries ." ,
    "" ,
    ]) ,
    llm=llm,
    verbose=True,
    tools=[search_engin_tool]   
    )

search_engine_task = Task(
 description="\n".join(["The task is  to  searsh for  products based on the  suggested searsh queries." ,
    "You have to collect results from each query and make a web searsh using the  tool " ,
    "Ignore any susbicious links or  not an ecommerce website link.",
    "Ignore any  searsh  result with score less then 60%" ,#the  score is  returned by the  tavily api
    "The searsh results will  be  used to  compare prices of  products from different websites",
    # "if the website is  sponsored don't include it"
    ]) ,
    expected_output="a JSON object containing the searsh results.",
    output_json=allSearshResults,
    output_file = os.path.join(output_dir , "step2_searsh_results.json"),
    agent=search_engine_agent,
  


)

### Agent 3

In [9]:
# from crewai_tools import ScrapeWebsiteTool

# # tool = ScrapeWebsiteTool()
# tool = ScrapeWebsiteTool(website_url='https://leprixmaroc.net/ps5-prix-maroc/' )

# # Extract the text from the site
# text = tool.run()
# print(text)

In [10]:
@tool
def scraping_tool(url: str , required_fields:list) -> str:
    """
    An AI tool to  help an agent  to  scrape  a web page

    Exemple :
     web_scraping_tool(url=url , required_fields=['product_title' , 'product_price' , 'product_image_url'])

    """

    details = scrape_client.smartscraper(
        website_url=url ,
        user_prompt="extract : ```json \n" +SingleSearshResult.schema_json()  + "``` From the web page"
    )

    return {
        "page_url": url ,
        "details":details
    }




scraping_agent = Agent(
    role="Web Scraping Agent",
    goal="To scrape and get data from any website.",
    backstory="\n".join(["The agent is  designed to  help in looking values from any website url" ,
    "these details  will be used to  decide woch best product to buy" ,
    ]) ,
    llm=llm,
    verbose=True,
    tools=[scraping_tool]   
    )

scraping_task = Task(
    description="\n".join(["The task is  to  extract products  details  from any ecommercestore page url",
    "the task has to  collect results from multiple pages urls",
    "if the parsed  data contain many products parsed dont include them in the output"
        ]) ,
    expected_output="a JSON object containing the scraping results.",
    output_json=AllExtractedProducts,
    output_file = os.path.join(output_dir , "step3_searsh_result.json"),
    agent=scraping_agent,   
)



### Agent 4

In [11]:
peocurement_report_author_agent = Agent(
    role="Peocurement Report Author Agent",
    goal="To Generate a professional Mardown file for the procurement repost ",
    backstory="""
    The agent is  designed to  assit in generationg a professionel Markdown page for the  procument report  report  after looking into  a list of peoducts.
    Your are also a good designer that in making  mardowns file in a sofisticat way .
    """,
    llm=llm,
    verbose=True,
    )



peocurement_report_author_task = Task(
    description="\n".join(["The task is  to generate a professional MArkdown file for the  procuremt  report.",
        "the  report  will includ the  search results and prices of  produts from differents  websites." ,
        "Ensure that all currency units (e.g., Dirhams, Dollars) are displayed clearly in the findings.",
        "Convert prices into a common currency (MAD) if possible ",
        "use the  provided context about the  company to  make a specializer report .",
        "The report should be structured with the following sections:",
        "1. Executive Summary: A brief overview of the procurement process and key findings.",
        "2. Introduction: An introduction to the purpose and scope of the report.",
        "3. Methodology: A description of the methods used to gather and compare prices.",
        "4. Findings: Detailed comparison of prices from different websites, including tables and charts. and  also mention the url of  each one",
        "5. Analysis: An analysis of the findings, highlighting any significant trends or observations.",
        "6. Recommendations: Suggestions for procurement based on the analysis.",
        "7. Conclusion: A summary of the report and final thoughts.",
        ]) ,
    expected_output="a Makdown file .",
    output_file = os.path.join(output_dir , "step4_procurement_report.md"),
    agent=peocurement_report_author_agent,   
)

### Crew

In [12]:
about_company= """ We are a company that provides AI Solutions to  help websites refine thier search ans recommendation engine based in Morocco , """

company_context = StringKnowledgeSource(
    content = about_company
)

In [13]:
crew = Crew(
    agents=[
    search_queries_recommender_agent ,
    search_engine_agent ,
    scraping_agent,
    peocurement_report_author_agent
    ],
    tasks=[
        search_queries_recommender_task
         , search_engine_task ,
         scraping_task ,
         peocurement_report_author_task
         ],
    process=Process.sequential,
    verbose=True ,  
    knowledge_sources=[company_context]
)

 
[2025-02-14 21:27:41][WARNING]: Failed to init knowledge: Please provide an OpenAI API key. You can get one at https://platform.openai.com/account/api-keys


In [14]:
crew.kickoff(
    inputs={"product_name": "Playstation5", "websites": ["jumia.ma" , "amazon.com"] , "country": "Morocco" , "language": "french"}
)

# Agent: Search Queries Recommender Agent
## Task: we are looking  to  buy Playstation5 at the  best prices (value for  a price strategy).
we target any if  these websites to  bue from : ['jumia.ma', 'amazon.com'] .
we want  to  reach all the available products on the internet to  be  compared later in another stage.the stores must  sell the  product in Morocco
the search keywords must be  in french languagethe search query must an ecommerse webpage for  product  , and not a blog or listing page.


# Agent: Search Queries Recommender Agent
## Final Answer: 
{
  "queries": [
    "acheter playstation 5 prix maroc jumia.ma",
    "playstation 5 meilleur prix maroc jumia.ma",
    "acheter ps5 pas cher maroc jumia.ma",
    "console ps5 prix maroc amazon.com"
  ]
}


# Agent: Search Engine Agent
## Task: The task is  to  searsh for  products based on the  suggested searsh queries.
You have to collect results from each query and make a web searsh using the  tool 
Ignore any susbicious links or

🖇 AgentOps: Failed to send events: API server: - internal server error
🖇 AgentOps: Failed to send events: API server: - internal server error
🖇 AgentOps: Failed to send events: API server: - internal server error


CrewOutput(raw='```markdown\n# Procurement Report: PlayStation 5 in Morocco\n\n## Executive Summary\n\nThis report analyzes the pricing and availability of the PlayStation 5 console in Morocco across various online retailers.  Our research focused on identifying the most competitive offers on Jumia.ma and other platforms.  Findings indicate price variations across retailers, with bundled offers presenting potential value.  We recommend prioritizing bundled deals on Jumia.ma for the best value, followed by exploring individual console options once pricing information becomes available.\n\n## Introduction\n\nThe purpose of this report is to compare prices and identify the best deals for procuring a PlayStation 5 console in Morocco.  This report will assist in making informed procurement decisions by providing a clear overview of available options and their associated costs.\n\n## Methodology\n\nWe conducted online searches on Jumia.ma and other relevant e-commerce websites using keywords